In [1]:
from itertools import combinations
from scipy.special import binom
import warnings
import sympy as sp
from sympy import Matrix, symbols, Symbol
warnings.simplefilter('ignore')

In [2]:
# Helper functions
def recursive_next_list_RBS(n, k, list_index, index):
    new_list_index = list_index.copy()
    new_list_index[index] += 1
    if (new_list_index[index] // n > 0):
        new_list_index = recursive_next_list_RBS(n - 1, k, new_list_index, index - 1)
        new_list_index[index] = new_list_index[index - 1] + 1
    return new_list_index

def dictionary_RBS(n, k):
    """ gives a dictionary that links the state and the list of active bits
    for a k arrangement basis """
    nbr_of_states = int(binom(n, k))
    RBS_dictionary = {}
    for state in range(nbr_of_states):
        if state == 0:
            RBS_dictionary[state] = [i for i in range(k)]
        else:
            RBS_dictionary[state] = recursive_next_list_RBS(n, k, RBS_dictionary[state - 1], k - 1)
    return RBS_dictionary

def map_RBS(n, k):
    """ Given the number of qubits n and the chosen Hamming weight k, outputs
    the corresponding state for a tuple of k active qubits. """
    Dict_RBS = dictionary_RBS(n, k)
    mapping_RBS = {tuple(val): key for (key, val) in Dict_RBS.items()}
    return mapping_RBS

def RBS_Unitary(nbr_state, gate_impact):
    """ Return an RBS corresponding unitary decomposed as coeffs that should be multiplied by
    cos(theta), coeffs that should be multiplied by sin(theta) and the ones that are constant
    equal to one. This decomposition allows to avoid inplace operations. 
    Args:
        - nbr_state: size of the considered basis
        - gate_impact: list of tuples of basis vectors. Their planar rotation satisfies 
        this transformation
    """
    cos_matrix = Matrix.zeros(nbr_state, nbr_state)
    sin_matrix = Matrix.zeros(nbr_state, nbr_state)
    id_matrix = Matrix.eye(nbr_state)
    for tuple_states in gate_impact:
        i, j = tuple_states
        id_matrix[i, i] = 0
        id_matrix[j, j] = 0
        cos_matrix[i, i] = 1
        cos_matrix[j, j] = 1
        sin_matrix[i, j] = 1
        sin_matrix[j, i] = -1
    return cos_matrix, sin_matrix, id_matrix

def RBS_generalized(a, b, n, k, mapping_RBS):
    """ Given the two qubits a,b the RBS gate is applied on, it outputs a list of
    tuples of basis vectors satisfying this transformation """
    # Selection of all the affected states
    RBS = []
    # List of qubits except a and b:
    list_qubit = [i for i in range(n)]
    list_qubit.pop(max(a, b))
    list_qubit.pop(min(a, b))
    # We create the list of possible active qubit set for this RBS:
    list_combinations = list(combinations(list_qubit, k - 1))
    for element in list_combinations:
        active_qubits_a = sorted([a] + list(element))
        active_qubits_b = sorted([b] + list(element))
        RBS.append((mapping_RBS[tuple(active_qubits_a)], mapping_RBS[tuple(active_qubits_b)]))
    return RBS

def RBS_Unitaries(n, k, list_gates):
    """ We store the RBS unitaries corresponding to each edge in the qubit connectivity to
    save memory. This allows to different RBS applied on the same pair of qubit to use the
    same unitary (but different parameters).
    Args:
        - n: number of qubits
        - k: chosen Hamming Weight
        - list_gates: list of tuples representing the qubits affected by each RBS
    Output:
        - RBS_Unitaries_dict: a dictionary with key tuples of qubits affected by RBS and
        with values tuples of tensors that decompose the equivalent unitary such as in
        RBS_Unitary (cos_matrix, sin_matrix, id_matrix)
    """
    RBS_Unitaries_dict, qubit_edges = {}, list(set(list_gates))
    mapping_RBS = map_RBS(n, k)
    for (i, j) in qubit_edges:
        RBS_Unitaries_dict[(i, j)] = RBS_Unitary(int(binom(n, k)), RBS_generalized(i, j, n, k, mapping_RBS))
    return RBS_Unitaries_dict

class RBS_Dense_density:
    """ This module describes the action of one RBS gate."""

    def __init__(self, qubit_tuple, theta_label):
        """ Args:
            - qubit_tuple: tuple of the 2 qubits index affected by the RBS
            - theta_label: label for the symbolic angle parameter theta
        """
        self.theta = sp.symbols(f'theta_{theta_label}')
        self.qubit_tuple = qubit_tuple

    def apply(self, input_matrix, RBS_unitaries):
        """ Application of the RBS corresponding unitary on the input state.
        Args:
            - input_matrix: a sympy Matrix representing the initial density operator.
            Its dimension is (nbr_batch, binom(I,2), binom(I,2)).
            - RBS_unitaries: a dictionary that gives the RBS unitary corresponding 
            to the qubit tuple such defined in RBS_Unitaries function. The unitary are
            of dimension (binom(I,2),binom(I,2))
        Output:
            - output state from the application of the RBS on the input state 
        """
        cos_theta = sp.cos(self.theta)
        sin_theta = sp.sin(self.theta)
        U = (RBS_unitaries[self.qubit_tuple][0] * cos_theta +
             RBS_unitaries[self.qubit_tuple][1] * sin_theta +
             RBS_unitaries[self.qubit_tuple][2])
        return U * input_matrix

class Dense_RBS_density_3D:
    """ This module describes the action of one RBS based VQC. """

    def __init__(self, I, J, k, list_gates):
        """ Args:
            - I: size of the square input image
            - list_gates: list of tuples representing the qubits affected by each RBS
        """
        self.RBS_Unitaries_dict = RBS_Unitaries(I+I+J, k, list_gates)
        self.RBS_gates = [RBS_Dense_density(list_gates[i], i+1) for i in range(len(list_gates))]

    def apply(self, input_state):
        """ Feedforward of the RBS based VQC.
        Arg:
            - input_state = a density operator on which is applied the RBS from the
            VQC. Its dimension is (nbr_batch, binom(2*I,2), binom(2*I,2))
        Output:
            - final density operator from the application of the RBS from the VQC on
            the input density operator. Its dimension is (nbr_batch, binom(2*I,2), binom(2*I,2)).
        """
        for RBS in self.RBS_gates:
            input_state = RBS.apply(input_state, self.RBS_Unitaries_dict)
        return input_state

def count_distinct_nonzero_nonone_elements(matrix):
    elements = matrix.tolist()
    flattened_elements = [item for sublist in elements for item in sublist]
    distinct_elements = set(flattened_elements)

    filtered_elements = {elem for elem in distinct_elements if elem != 0 and elem != 1}

    return len(filtered_elements)

I = 1
J = 1
k = 1
N = int(binom(2*I+J, k))
x = sp.eye(N)
# display(x)
list_gates = [(0,1),(1,2),(0,1)]
# list_gates_full = [(i, j) for i in range(2*I+J) for j in range(2*I+J) if i != j]
dense = Dense_RBS_density_3D(I, J, k, list_gates)
out = dense.apply(x)
out
# out = dense.apply(x)[-10:, :]
# print("max: " + str(N*N))
# count_distinct_nonzero_nonone_elements(out)

Matrix([
[-sin(theta_1)*sin(theta_3)*cos(theta_2) + cos(theta_1)*cos(theta_3),  sin(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2), sin(theta_2)*sin(theta_3)],
[-sin(theta_1)*cos(theta_2)*cos(theta_3) - sin(theta_3)*cos(theta_1), -sin(theta_1)*sin(theta_3) + cos(theta_1)*cos(theta_2)*cos(theta_3), sin(theta_2)*cos(theta_3)],
[                                          sin(theta_1)*sin(theta_2),                                          -sin(theta_2)*cos(theta_1),              cos(theta_2)]])